In [ ]:
!python -m spacy download en
!pip install transformers
!pip install torch
!pip install tensorflow
!python -m spacy download en_core_web_lg

In [ ]:
import numpy as np
import re
import networkx as nx
import spacy
from summa import keywords
from summa.summarizer import summarize

# Laden des Spacy-Modells
import evaluate
import nltk
from nltk.tokenize import sent_tokenize
from transformers import AutoModelForSeq2SeqLM, PegasusForConditionalGeneration, PegasusTokenizer, AutoTokenizer
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import math
from transformers import PreTrainedTokenizerFast
from tqdm import tqdm
import random
from transformers import pipeline
import torch

In [ ]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print(x)
else:
    print ("MPS device not found.")

In [ ]:
#model_name='facebook/bart-large-cnn'
model_name='facebook/bart-large'
#model_name='NICFRU/bart-base-paraphrasing'
tokenizer = AutoTokenizer.from_pretrained(model_name)
summarizer = pipeline("text2text-generation", model=model_name,device='mps:0')

In [ ]:

def reduce_repetitions(text):
    text = re.sub(r'\.{2,}', '.', text)
    text = re.sub(r'\!{2,}', '!', text)
    text = re.sub(r'\,{2,}', ',', text)
    text = re.sub(r'\;{2,}', ';', text)
    return text




def textrank_extractive(text, compression_rate=0.5,split='\. '):
    # Tokenisierung
    nlp = spacy.load("en_core_web_lg")
    #doc = nlp(text.replace("\n\n", " "))

    # Split the text at each ". " that is not followed by a single letter
    doc = re.split(fr'(?<!\b\w\w){split}', reduce_repetitions(re.sub(' +', ' ', text.replace("\n", " ").replace('-',' ').replace('_',' ').replace("\'", "").replace("!", ".").replace("?", ".").replace(";", ""))))
    sentences = [sent for sent in doc if len(sent.replace("-", " ").split()) > 2]

    # Speichern der Spacy-Dokumente der Sätze für spätere Verwendung
    sentence_docs = [nlp(sentence) for sentence in sentences]

    # Extrahiere Schlüsselsätze mit TextRank
    num_sentences = max(1, int(len(sentences) * compression_rate))
    extracted_sentences = summarize(text, words=num_sentences, split=True)

    # Erzeuge eine Matrix mit den Ähnlichkeiten zwischen den Sätzen
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
    for i, doc_i in enumerate(sentence_docs):
        for j, doc_j in enumerate(sentence_docs):
            similarity = similarity_function(doc_i, doc_j)
            similarity_matrix[i, j] = similarity
    
    # Konstruiere einen Graphen mit den Sätzen als Knoten und den Ähnlichkeiten als Kanten
    graph = nx.from_numpy_array(similarity_matrix)

    # Berechne den TextRank-Score für jeden Satz
    scores = nx.pagerank_numpy(graph)

    # Wähle die besten Sätze basierend auf ihren TextRank-Scores aus
    top_sentences = sorted(scores, key=scores.get, reverse=True)[:num_sentences]

    # Sortiere die ausgewählten Sätze nach ihrer Position im Text
    top_sentences = sorted(top_sentences)

    # Gebe die extrahierten Schlüsselsätze zurück
    extracted_sentences = [sentences[index] for index in top_sentences]
    return extracted_sentences


def similarity_function(doc1, doc2):
    # Berechne die Cosinus-Ähnlichkeit zwischen den beiden Dokumenten
    similarity = doc1.similarity(doc2)
    return similarity




def compression_ratio(text, summary):
    # Berechne das Verhältnis der Anzahl der Wörter in der Zusammenfassung zur Anzahl der Wörter im Ausgangstext
    num_words_text = len(text.split())
    num_words_summary = len(summary.split())
    ratio = num_words_summary / num_words_text
    return ratio



def compression(text, compression_rate,split='\. '):
    max_iterations = 20
    iterations = 0
    #compression_rate -= 0.05
    
    extracted = textrank_extractive(text, compression_rate,split)
    summary = '. '.join(extracted)
    compression_rate_renwed = compression_rate


    while compression_ratio(text, summary) < compression_rate and iterations < max_iterations:
        iterations += 1
        compression_rate_renwed += 0.05
        if compression_rate_renwed > 1:
            compression_rate_renwed = 1
        extracted = textrank_extractive(text, compression_rate=compression_rate_renwed)
        summary = '. '.join(extracted)
    return summary




In [ ]:
def token_count(text):
    tokens = text.split()
    return len(tokens)


def adjust_length(text):
    length = token_count(text)
    if length <20:
        min_length = length + int(length * 0.05)
        max_length = min_length +min_length
    elif length <50:
        min_length = length + int(length * 0.05)
        max_length = min_length +min_length* 0.5
    elif length <60:
        min_length = length + int(length * 0.05)
        max_length = min_length +min_length* 0.4
    elif length < 80:
        min_length = length + int(length * 0.05)
        max_length = min_length + min_length* 0.25
    elif length < 100:
        min_length = length + int(length * 0.3)
        max_length = min_length + 100
    else:
        min_length = math.ceil(length / 50) * 70
        max_length = min_length + 100
    return min_length, max_length

In [ ]:
def batch_sent(sentenc,splitt=180,split='\. '):
    
    sentences = re.split(fr'(?<!\b\w\w){split}', sentenc.lower())

    # Erstellen Sie Batches von Sätzen, die weniger als 1024 Tokens enthalten
    batches = []
    batch = []
    batch_len = 0
    for sentence in sentences:
        
        sentence_len = len(tokenizer.tokenize(sentence))
        if sentence_len + batch_len > splitt:
            if sentence_len < splitt:  # überspringen Sie Sätze, die länger als 256 Tokens sind
                batches.append(batch)
                batch = [sentence]
                batch_len = sentence_len
            # wenn ein Satz alleine 1024 Tokens überschreitet, wird er übersprungen
        else:
            batch.append(sentence)
            batch_len += sentence_len
    batches.append(batch)
    return batches

In [ ]:
def paraphrase_of_text(df_s,text_name='text',komp_name='reduction_multiplier',split='\. '):

    df_summary_testing = pd.DataFrame(columns=['Zusammenfassung','Min_Kompressionsrate', 'Max_Kompressionsrate', 'Endgueltige_Kompressionsrate','länge Zusammenfassung','länge Ausgangstext','batch_texts','batch_output'])
    # Teilen Sie den Text in Sätze --> für tatsächliche umsetzubng
    soplitting=True
    
    for _, row in tqdm(df_s.iterrows(), total=df_s.shape[0]):
        batch_text_list=[]

        output_text_list=[]
        text = row[text_name]
        komp = row[komp_name]
        text_gesamt_list=[]

        for batch in tqdm(batch_sent(text,split=split), desc='Verarbeite Batches'):
            # Zusammenfügen der Sätze in einem Batch
            #print(batch)
            if len(batch):
                #print('Läuft')
                batch_text = '. '.join(batch)
                batch_text += "."
                batch_text_list.append(batch_text)
                min_length_test, max_length_test = adjust_length(batch_text)
                ext_summary=summarizer(batch_text, max_length=int(round(max_length_test*komp,0)), min_length=int(round(min_length_test*komp,0)),length_penalty=100,num_beams=2)
            # Erstellen Sie einen read_csv für die aktuellen Ergebnisse

            text_gesamt_list.append(ext_summary[0]['generated_text'])
         
        text_gesamt = '. '.join(text_gesamt_list)
        actual_compression_rate = len(text_gesamt.split(' '))/len(text.split(' '))*100
      
        df_current = pd.DataFrame({
            'Zusammenfassung': [text_gesamt],
            'Min_Kompressionsrate': [min_length_test],
            'Max_Kompressionsrate': [max_length_test],
            'Endgueltige_Kompressionsrate': [actual_compression_rate],
            'länge Zusammenfassung': [len(text_gesamt.split(' '))],
            'länge Ausgangstext': [len(text.split(' '))],
            'batch_texts': [batch_text_list],
            'batch_output': [text_gesamt_list]
        
        })
       
        # Fügen Sie die Daten zum DataFrame hinzu
        df_summary_testing = pd.concat([df_summary_testing, df_current], ignore_index=True)
      
    return df_summary_testing


In [ ]:
def calculate_compression(df, total_tokens_col, current_tokens_col, desired_compression_rate):
    df['current_compression_rate'] = df[current_tokens_col] / df[total_tokens_col]
    df['compression_difference'] = df[desired_compression_rate] - df['current_compression_rate']
    df['reduction_multiplier'] = df[desired_compression_rate] / df['current_compression_rate']
    return df

In [ ]:
def text_rank_algo(df,seed=10,split='\. ',random_T=True,column='text'):
    df_return = pd.DataFrame(columns=['text','text_rank_text','tokens_gesamt', 'token_text_rank','desired_compression_rate','text_rank_compression_rate'])
    # Schleife zur Generierung der zufälligen Werte
    random.seed(seed)
    for index, row in tqdm(df.iterrows()):
        text = row[column].replace("\n", " ")
        if random_T:
            random_value = round(random.uniform(0.2, 0.8), 2)  # Zufälliger Wert zwischen 0.2 und 0.8 auf zwei Stellen nach dem Komma begrenzt
        else:
            if row['reduction_multiplier']<0.8:
                random_value=row['desired_compression_rate']
            elif row['reduction_multiplier']<0.9:
                random_value=row['reduction_multiplier']
            else:
                random_value=1
        text_rank_text= compression(text.replace("\n\n", " "),random_value,split)
        compression_ratio_value=compression_ratio(text, compression(text,random_value,split))
        text=re.sub(' +', ' ', text.replace("\n", " ").replace('-',' ').replace('_',' ').replace("\'", "").replace("!", ".").replace("?", ".").replace(";", ""))
        text_rank_text=re.sub(' +', ' ', text_rank_text.replace("\n", " ").replace('-',' ').replace('_',' ').replace("\'", "").replace("!", ".").replace("?", ".").replace(";", ""))
        df_current = pd.DataFrame({
                'text':[text],
                'text_rank_text': [text_rank_text],
                'tokens_gesamt': [len(text.split(' '))],
                'token_text_rank': [len(text_rank_text.split(' '))],
                'desired_compression_rate': [random_value],
                'text_rank_compression_rate': [compression_ratio_value]
            })
            # Fügen Sie die Daten zum DataFrame hinzu
        df_return = pd.concat([df_return, df_current], ignore_index=True)
    return df_return


In [ ]:
def execute_text_gen(df,split='\. ',seed=10):
    rank_df=text_rank_algo(df,seed=seed,split=split)
    df_zwischen=calculate_compression(rank_df, 'tokens_gesamt', 'token_text_rank', 'desired_compression_rate')
    df_sum=paraphrase_of_text(df_zwischen[['text_rank_text','reduction_multiplier']],text_name='text_rank_text',split=split)
    merged_df = pd.concat([df_sum, df_zwischen], axis=1)
    merged_df['ent_com_rate']=merged_df['länge Zusammenfassung'] / merged_df['tokens_gesamt']
    new_df=calculate_compression(merged_df[['Zusammenfassung','länge Zusammenfassung','text','tokens_gesamt','desired_compression_rate','ent_com_rate']], 'tokens_gesamt', 'länge Zusammenfassung', 'desired_compression_rate')
    df_testen=text_rank_algo(new_df,random_T=False,column = 'Zusammenfassung')
    a=pd.concat([df_testen[['text_rank_text','token_text_rank',]], new_df[['Zusammenfassung','länge Zusammenfassung','text','tokens_gesamt','desired_compression_rate','ent_com_rate'	]]], axis=1)
    a['ent_com_rate']=a['länge Zusammenfassung'] / a['tokens_gesamt']
    return a